In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import datetime

In [2]:
url = "https://bringatrailer.com/listing/2003-mitsubishi-lancer-evolution-41/"
url = "https://bringatrailer.com/listing/2005-subaru-impreza-wrx-sti-85/"

In [3]:

pd_db = pd.DataFrame()

# Pandas DF column names
COLUMNS = ['make', 'model', 'year', 'kilometers', 'miles', 'mileage notes', 
           'sale_status', 'final bid price', 'auction date', 'vin', 'details', 'url']

NEEDED_COLUMNS = ['make', 'model', 'year', 'miles', 'color', 'auction_year', 'engine_size', 'cylinders']

In [2]:
url = "https://bringatrailer.com/listing/1997-porsche-911-carrera-4s-62/"
# url = "https://bringatrailer.com/listing/2000-subaru-impreza-2-5rs-23/"
# url = "https://bringatrailer.com/listing/2007-subaru-impreza-wrx-sti-5/"

In [3]:
html = requests.get(url)
# Pull html data
vehicle_data_soup = BeautifulSoup(html.content, 'html.parser')
auction_result_string = vehicle_data_soup.find("h1", class_ = 'post-title')

In [4]:
words = auction_result_string.text
model2 = ""

for idx, word in enumerate(words.split()):
    if word.isdigit():
        # print(word, idx)
        model2 = " ".join(words.split()[idx+1:])
        break
model2

'Porsche 911 Carrera 4S Coupe 6-Speed'

In [5]:
try:
    post_title_soup = vehicle_data_soup.find("h1", class_ = 'post-title').text
    post_title = ""

    for idx, word in enumerate(post_title_soup.split()):
        if word.isdigit():
            post_title = " ".join(post_title_soup.split()[idx+1:]).strip(' ')
            break

except:
    post_title = ""

post_title.lower()

'porsche 911 carrera 4s coupe 6-speed'

In [4]:
# get vehicle html
try:
    html = requests.get(url)
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print("URL is not correct")

# Pull html data
vehicle_data_soup = BeautifulSoup(html.content, 'html.parser')

# Get image
image = vehicle_data_soup.find_all('img', class_='post-image')[0]['src']
print(image)

# Parse html and get auction data - "Sold for $XXXX on XX/XX/XXXX"

auction_result_string = vehicle_data_soup.find("span", class_ = 'listing-available-countdown')

# Get auction year and price
if auction_result_string:
    print('Auction is ongoing..')
    # Get vehicle sale date
    auction_year = datetime.datetime.now().year
    # print(auction_year)
    # Get current vehicle price
    auction_current_price = vehicle_data_soup.find("strong", class_ = 'info-value').text
    print(auction_current_price)
    sale_price = int(re.findall('[0-9]+,[0-9]+', auction_current_price)[0].replace(",",""))
    # print(sale_price)

else:
    auction_result_string = vehicle_data_soup.find("span", class_ = 'info-value').text
    print('Auction is over')
    sale_date = auction_result_string.split()[-1]
    auction_year = (datetime.datetime.strptime(sale_date, '%m/%d/%y').year)
    # print(auction_year)
    # Get vehicle sale price
    sale_price = int(re.findall('[0-9]+,[0-9]+', auction_result_string)[0].replace(",",""))
    # print(sale_price)

# Get Make & Model
vehicle_make, vehicle_model = [s.text for s in vehicle_data_soup.find_all("button", class_ = 'group-title')][0:2]
vehicle_make = vehicle_make.replace('Make', '').strip(' ')
vehicle_model = vehicle_model.replace('Model', '').replace(vehicle_make, '').strip(' ')

print(vehicle_make, vehicle_model, sale_price)

https://bringatrailer.com/wp-content/uploads/2022/10/2005_subaru_impreza-wrx-sti_img_3727-7-31397.jpg?fit=940%2C626
Auction is over
Subaru WRX STi 22000


In [5]:
print(auction_result_string)

<span class="listing-available-countdown" data-countdown="44758470" data-until="1669235400"></span>


In [6]:
# Get listing details - vin, miles, etc. - second entry
listing_details_html = vehicle_data_soup.find_all("div", class_="item")



# Find the listing_details using keyword "Listing Details"
for detail in listing_details_html:
    if detail.find("strong") and "Listing Details" in detail.find("strong"):
        listing_details = detail
        break

        
        
# Extract details from html "li"
results = None
results = [detail.text for detail in listing_details.find_all('li')]


# Get vehicle mileage, normally second entry in "results"
is_mileage_units = True
mileage_words = ['Miles', 'miles', 'Mile', 'mile'] 
kilometer_words = ['Kilometers', 'kilometers', 'Kilometer', 'kilometer', 'KM', 'km']
for result in results:
    result = result.replace("-"," ")
    if any(word in result for word in kilometer_words):
        vehicle_mileage_notes = result
        is_mileage_units = False
        break
    elif any(word in result for word in mileage_words):
        vehicle_mileage_notes = result
        break

        
# Extract Mileage figure from vehicle_mileage_notes
if vehicle_mileage_notes is None:
    vehicle_mileage = None
else:
    vehicle_mileage = re.findall('[0-9]+,[0-9]+', vehicle_mileage_notes)
    if len(vehicle_mileage) < 1: # If no match is found, mileage must contain 'k' at end i.e., 47k miles
        vehicle_mileage = re.findall('[0-9]+[kK]', vehicle_mileage_notes)
        if len(vehicle_mileage) >= 1: # If match is found, strip k from end
            vehicle_mileage = [vehicle_mileage[0].strip('k').strip('K') + ',000']
    if len(vehicle_mileage) < 1: # Still no match found, try mileage < 1,000
        vehicle_mileage = re.findall('[0-9]+', vehicle_mileage_notes)
    if vehicle_mileage == []:
        vehicle_mileage = None
vehicle_mileage = vehicle_mileage if type(vehicle_mileage) is not list else vehicle_mileage[0]

# Check if units are in miles or km and make adjustments if needed
if vehicle_mileage is not None:
    vehicle_mileage = int(vehicle_mileage.replace(',',''))
    if is_mileage_units: # units are mileage
        vehicle_kilometers = int(1.60934 * vehicle_mileage)
    else: # units are km
        vehicle_kilometers = vehicle_mileage
        vehicle_mileage = int(0.621371 * vehicle_mileage)
        
      
    
print(vehicle_mileage)
print(results)

51000
['Chassis: JA3AH86F73U088113', '51k Miles', 'Turbocharged 2.0-Liter Inline-Four', 'Five-Speed Manual Transaxle ', 'Viscous Center Differential', 'Limited-Slip Rear Differential', 'Rally Red Paint', 'Black & Blue Cloth Upholstery', '17" Enkei Wheels', 'HID Headlights', 'HKS Camshafts', 'Boostlabs Turbocharger', 'E85 ECU Tune', 'Recaro Front Seats', 'Air Conditioning', 'Sony Stereo', 'Carfax report']


In [7]:
colors = ["white", "black", "gray", "silver", "blue", "red", "brown", "green", "orange", "beige", "purple", "gold", "yellow"]
paint_string = None
paint_color = None

for result in results:
    if paint_string:
        break
    results_separated = result.split(",")
    results_separated = [result.strip().lower() for detail in results_separated]
    
    for detail in results_separated:
        if "paint" in detail or any(color in detail for color in colors):
            paint_string = detail
            break
    
for word in paint_string.split():
    if any(color in word for color in colors):
        paint_color = word


print(paint_color)

red


In [8]:
# Get vehicle model year
model_year_text = vehicle_data_soup.find("h1", class_ = 'post-title').text
model_year = re.findall("(\d{4})", model_year_text)

############ NOT NEEDED ################
# Check sale status - i.e., sold or not
sale_status = "Not Sold"
if "Sold" in auction_result_string or "sold" in auction_result_string:
    sale_status = "Sold"
#######################################
    
print(model_year[0])


2003


In [9]:
# Get vehicle sale date
# sale_date = auction_result_string.split()[-1]
# print(sale_date)
# auction_year = (datetime.datetime.strptime(sale_date, '%m/%d/%y').year)

# Get vehicle sale price
# sale_price = int(re.findall('[0-9]+,[0-9]+', auction_result_string)[0].replace(",",""))

# Get vehicle vin, first entry in "results"
vehicle_vin = results[0].split()[-1]

# # Combine results into 1 string
# joined_results = " ,".join(results)


print(vehicle_make, vehicle_model, model_year[0], vehicle_kilometers, vehicle_mileage, vehicle_mileage_notes, sale_status, auction_year, vehicle_vin, joined_results, url)

NameError: name 'joined_results' is not defined

## Extracting Engine Size and # Cylinders

In [14]:
# Split the remaining details data
# results_pd = pd.DataFrame()
# results_pd['details'] = joined_results
# split_results = results_pd.details.str.split(',', expand=False)
# Combine results into 1 string
# joined_results = " ,".join(results)
# split_results = joined_results.split(',')
# split_results = results

# split_results = split_results.str.lower()
# Extract engine size from details
eng_keywords = ["liter", "v6", "v8", "engine", "inline", "three", "four", "five", "six", "eight", "ci", "cc", "flathead", "cylinder", "dohc", "sohc", "ohc", "turbocharged"]
eng_size_keywords = ["liter", "ci", "cc"]
eng_size_re = ["[0-9]+.[0-9]+", "[0-9]+.[0-9]+l"]
# Best number finding regex ever!
match = False
engine_size_string = None

for details in results:
    # print(details)
    if match:
        break
    for detail in details.split():
        detail=detail.lower()
        # print(detail)
        if any(word in detail for word in eng_keywords):
            match = True
            if len(re.findall(eng_size_re[0], detail)) > 0 or len(re.findall(eng_size_re[1], detail)) > 0:
                engine_size_string = detail
            elif any(w in detail for w in eng_size_keywords):
                engine_size_string = detail
                print('this')

                # engine_size_string = details
                # print(detail)
                break


print(engine_size_string)

2.0-liter


In [11]:
numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
rx = re.compile(numeric_const_pattern, re.VERBOSE)
engine_size = None

match = rx.findall(engine_size_string)

# Convert cc or ci to liters
if "cc" in engine_size_string or "cubic centimeters" in engine_size_string:
    size = engine_size_string.replace("cc", "")
    size = engine_size_string.replace("cubic centimeters", "")
    # print('cc found', size)
    engine_size = (float(size)/1000)

elif "ci" in engine_size_string or "c.i." in engine_size_string:
    size = engine_size_string.replace("ci", "")
    size = engine_size_string.replace("c.i.", "")
    engine_size = (float(size)*0.0163871)

else:
    if float(match[0]) > 12:
        engine_size = 2.0
    else:
        engine_size = (float(match[0]))
        
engine_size

2.0

In [33]:
cylinder_keywords = ["inline", "cylinder", "cyl","two", "three", "four", "five", "six", "eight", "v4", "v6", "v8", "v10", "v12", "v-4", "v-6", "vr6", "v-8", "v-10", "v-12", "w12", "w-12", "flat4", "flat-4", "flat 4", "flat6", "flat 6", "flat-6"]
cylinder_re = [""]
singles = ["1", "single"]
ones = ["1", "one"]
twins = ["2", "twin"]
triples = ["3", "triple"]
twos = ["2", "two"]
threes = ["3", "three"]
fours = ["4", "four"]
fives = ["5", "five"]
sixes = ["6", "six"]
eights = ["8", "eight"]
tens = ["10", "ten"]
twelves = ["12", "twelve"]

num_cyl_string = None
match = False

# Extract num of cylinders
for sentence in split_results:
    if match:
        break
    words = sentence.lower().split(" ")
    for word in words:
        if any(w in word for w in cylinder_keywords):
            num_cyl_string = word
            match = True
            break

num_cylinders = None

# Extract int from cylinder str
for k in [threes, fours, fives, sixes, eights, tens, twelves, ones, twos, singles, twins, triples]:
    if k[0] in num_cyl_string or k[1] in num_cyl_string:
        num_cylinders = int(k[0])
        match = True
        break

        
print(num_cylinders)


4


In [45]:
vehicle_dict = {
    "make": vehicle_make,
    "model": vehicle_model,
    "year": model_year[0],
    "miles": vehicle_mileage,
    "color": paint_color,
    "auction_year": auction_year,
    "engine_size": engine_size,
    "cylinders": num_cylinders
}

'Mitsubishi'